In [ ]:
# -*- coding: utf-8 -*-
# 文件名: solve_q1_with_full_sensitivity.py (v3.0 - 包含全部三类灵敏度分析)

import pandas as pd
import pyomo.environ as pyo
import os
import time
import re
import numpy as np
import copy
import warnings
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

# --- 数据加载函数 (与之前相同，为完整性保留) ---
def load_and_prepare_data(data_path_f1, data_path_f2):
    """
    数据加载与处理函数。
    """
    try:
        # print("正在读取Excel文件...")
        plots_df = pd.read_excel(data_path_f1, sheet_name='乡村的现有耕地')
        crops_info_df = pd.read_excel(data_path_f1, sheet_name='乡村种植的农作物')
        stats_df_detailed = pd.read_excel(data_path_f2, sheet_name='2023年统计的相关数据')
        past_planting_df = pd.read_excel(data_path_f2, sheet_name='2023年的农作物种植情况')
        # print(" -> Excel文件读取成功。")
    except Exception as e:
        print(f"错误: 读取Excel文件失败。具体错误: {e}")
        return None
    for df in [plots_df, crops_info_df, stats_df_detailed, past_planting_df]:
        df.columns = df.columns.str.strip()
    params = {}
    params['I_plots'] = plots_df['地块名称'].tolist()
    params['P_area'] = dict(zip(plots_df['地块名称'], plots_df['地块面积/亩']))
    params['P_plot_type'] = dict(zip(plots_df['地块名称'], plots_df['地块类型']))
    params['J_crops'] = crops_info_df['作物名称'].unique().tolist()
    params['P_crop_type'] = dict(zip(crops_info_df['作物名称'], crops_info_df['作物类型']))
    bean_keywords = ['豆', '豆类']
    params['J_bean'] = [j for j, ctype in params['P_crop_type'].items() if isinstance(ctype, str) and any(keyword in ctype for keyword in bean_keywords)]
    params['P_past'] = {(i, j): 0 for i in params['I_plots'] for j in params['J_crops']}
    for _, row in past_planting_df.iterrows():
        if row['种植地块'] in params['I_plots'] and row['作物名称'] in params['J_crops']:
            params['P_past'][row['种植地块'], row['作物名称']] = 1
    def clean_and_convert(value):
        if isinstance(value, str) and ('-' in value or '–' in value or '—' in value):
            parts = re.split(r'[-–—]', value.strip())
            try:
                if len(parts) == 2: return (float(parts[0]) + float(parts[1])) / 2
            except ValueError: return pd.NA
        try: return float(value)
        except (ValueError, TypeError): return pd.NA
    for col in ['亩产量/斤', '种植成本/(元/亩)', '销售单价/(元/斤)']:
        if col in stats_df_detailed.columns:
            stats_df_detailed[col] = stats_df_detailed[col].apply(clean_and_convert)
            stats_df_detailed[col] = pd.to_numeric(stats_df_detailed[col], errors='coerce')
    stats_df_detailed.dropna(inplace=True)
    default_yield = stats_df_detailed['亩产量/斤'].median() / 2 if not stats_df_detailed.empty else 500
    default_cost = stats_df_detailed['种植成本/(元/亩)'].median() if not stats_df_detailed.empty else 500
    default_price = stats_df_detailed['销售单价/(元/斤)'].median() * 2 if not stats_df_detailed.empty else 10
    params['P_yield'], params['P_cost'], params['P_price'] = {}, {}, {}
    for _, row in stats_df_detailed.iterrows():
        key = (row['作物名称'], row['地块类型'])
        params['P_cost'][key] = row['种植成本/(元/亩)']
        params['P_yield'][key] = row['亩产量/斤'] / 2
        params['P_price'][key] = row['销售单价/(元/斤)'] * 2
    missing_combinations = 0
    for crop in params['J_crops']:
        for plot_type in plots_df['地块类型'].unique():
            key = (crop, plot_type)
            if key not in params['P_yield']:
                params['P_yield'][key], params['P_cost'][key], params['P_price'][key] = default_yield, default_cost, default_price
                missing_combinations += 1
    if missing_combinations > 0: warnings.warn(f"警告: 为{missing_combinations}个作物-地块类型组合使用了默认值")
    params['P_demand'] = {j: 0 for j in params['J_crops']}
    temp_planting_details = pd.merge(past_planting_df, plots_df, left_on='种植地块', right_on='地块名称')
    for j in params['J_crops']:
        total_yield_j = 0
        crop_plantings = temp_planting_details[temp_planting_details['作物名称'] == j]
        for _, planting_row in crop_plantings.iterrows():
            plot_type, area = planting_row['地块类型'], planting_row['种植面积/亩']
            key = (j, plot_type)
            if key in params['P_yield']: total_yield_j += params['P_yield'][key] * area
        params['P_demand'][j] = total_yield_j if total_yield_j > 0 else 1000
    params['S_suitability'] = {}
    for i in params['I_plots']:
        plot_t = params['P_plot_type'].get(i)
        if not plot_t: continue
        for j in params['J_crops']:
            crop_t = params['P_crop_type'].get(j)
            if not isinstance(crop_t, str): continue
            is_bean = j in params['J_bean']
            for k in [1, 2]:
                suitable = 0
                if plot_t in ['平旱地', '梯田', '山坡地'] and ('粮食' in crop_t or is_bean) and k == 1: suitable = 1
                elif plot_t == '水浇地':
                    if (crop_t == '水稻' and k == 1) or ('蔬菜' in crop_t): suitable = 1
                elif plot_t == '普通大棚' and (('蔬菜' in crop_t and k == 1) or ('食用菌' in crop_t and k == 2)): suitable = 1
                elif plot_t == '智慧大棚' and ('蔬菜' in crop_t): suitable = 1
                params['S_suitability'][i, j, k] = suitable
    print(" -> 数据参数准备完成。")
    return params

# --- 1. 修改 build_model 以接受 A_min ---
def build_model(params, case_type, A_min=0.1):
    """构建优化模型 (A_min 可配置)"""
    model = pyo.ConcreteModel(f"Q1_Model_{case_type}")
    model.I = pyo.Set(initialize=params['I_plots'])
    model.J = pyo.Set(initialize=params['J_crops'])
    model.Y = pyo.Set(initialize=list(range(2024, 2031)))
    model.K = pyo.Set(initialize=[1, 2])
    model.x = pyo.Var(model.I, model.J, model.K, model.Y, domain=pyo.NonNegativeReals, bounds=(0, 150))
    model.u = pyo.Var(model.I, model.J, model.K, model.Y, domain=pyo.Binary)
    model.z = pyo.Var(model.I, model.J, model.Y, domain=pyo.Binary)
    model.Sales = pyo.Var(model.J, model.K, model.Y, domain=pyo.NonNegativeReals)
    if case_type == 'discount':
        model.OverSales = pyo.Var(model.J, model.K, model.Y, domain=pyo.NonNegativeReals)
    def objective_rule(m):
        revenue = sum(params['P_price'].get((j, params['P_plot_type'].get(i)), 0) * m.Sales[j,k,y] for i in m.I for j in m.J for k in m.K for y in m.Y)
        if case_type == 'discount':
            revenue += sum(0.5 * params['P_price'].get((j, params['P_plot_type'].get(i)), 0) * m.OverSales[j,k,y] for i in m.I for j in m.J for k in m.K for y in m.Y)
        total_cost = sum(params['P_cost'].get((j, params['P_plot_type'].get(i)), 0) * m.x[i,j,k,y] for i in m.I for j in m.J for k in m.K for y in m.Y)
        return revenue - total_cost
    model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)
    def dispersion_rule(m, j, k, y): return sum(m.u[i, j, k, y] for i in m.I) <= 10
    model.dispersion_con = pyo.Constraint(model.J, model.K, model.Y, rule=dispersion_rule)
    def production_rule(m, j, k, y):
        total_prod = sum(params['P_yield'].get((j, params['P_plot_type'][i]), 0) * m.x[i, j, k, y] for i in m.I)
        if case_type == 'waste': return m.Sales[j, k, y] <= total_prod
        else: return m.Sales[j, k, y] + m.OverSales[j, k, y] == total_prod
    model.prod_con = pyo.Constraint(model.J, model.K, model.Y, rule=production_rule)
    def demand_rule(m, j, k, y): return m.Sales[j, k, y] <= params['P_demand'].get(j, 0)
    model.demand_con = pyo.Constraint(model.J, model.K, model.Y, rule=demand_rule)
    def area_rule(m, i, k, y): return sum(m.x[i, j, k, y] for j in m.J) <= params['P_area'][i]
    model.area_con = pyo.Constraint(model.I, model.K, model.Y, rule=area_rule)
    def suitability_rule(m, i, j, k, y):
        if params['S_suitability'].get((i, j, k), 0) == 0: return m.x[i, j, k, y] == 0
        return pyo.Constraint.Skip
    model.suitability_con = pyo.Constraint(model.I, model.J, model.K, model.Y, rule=suitability_rule)
    def u_link_upper_rule(m, i, j, k, y): return m.x[i, j, k, y] <= params['P_area'][i] * m.u[i, j, k, y]
    model.u_link_upper = pyo.Constraint(model.I, model.J, model.K, model.Y, rule=u_link_upper_rule)
    # 使用传入的 A_min
    def u_link_lower_rule(m, i, j, k, y): return m.x[i, j, k, y] >= A_min * m.u[i, j, k, y]
    model.u_link_lower = pyo.Constraint(model.I, model.J, model.K, model.Y, rule=u_link_lower_rule)
    def z_link_rule(m, i, j, y): return m.z[i, j, y] >= sum(m.u[i, j, k, y] for k in m.K)
    model.z_link_con = pyo.Constraint(model.I, model.J, model.Y, rule=z_link_rule)
    def rotation_past_rule(m, i, j):
        if params['P_past'].get((i, j), 0) > 0: return m.z[i, j, 2024] <= 0
        return pyo.Constraint.Skip
    model.rotation_past_con = pyo.Constraint(model.I, model.J, rule=rotation_past_rule)
    def rotation_future_rule(m, i, j, y):
        if y < 2030: return m.z[i, j, y] + m.z[i, j, y + 1] <= 1
        return pyo.Constraint.Skip
    model.rotation_future_con = pyo.Constraint(model.I, model.J, model.Y, rule=rotation_future_rule)
    model.bean_con = pyo.ConstraintList()
    for i in model.I:
        if i in params['P_plot_type']:
            past_bean = sum(params['P_past'].get((i, j), 0) for j in params['J_bean'])
            model.bean_con.add(past_bean + sum(model.z[i, j, y] for j in params['J_bean'] for y in [2024, 2025]) >= 1)
            for y_start in range(2024, 2031 - 2):
                model.bean_con.add(sum(model.z[i, j, y] for j in params['J_bean'] for y in range(y_start, y_start + 3)) >= 1)
    return model

# --- 2. 修改 solve_for_profit 以接受 A_min ---
def solve_for_profit(params, case_type, solver_path, timeout=120, A_min=0.1):
    """“安静”的求解器，接受 A_min 参数。"""
    model = build_model(params, case_type, A_min=A_min) # 将 A_min 传递给模型
    solver = pyo.SolverFactory('cbc', executable=solver_path)
    solver.options['sec'] = timeout
    try:
        results = solver.solve(model, tee=False)
        if (results.solver.termination_condition in [pyo.TerminationCondition.optimal, pyo.TerminationCondition.feasible, pyo.TerminationCondition.maxTimeLimit]):
            if pyo.value(model.profit, exception=False) is not None:
                return pyo.value(model.profit)
    except Exception as e:
        print(f"灵敏度分析中求解出错: {e}")
    return None

# --- 市场参数灵敏度分析函数 (与之前相同) ---
def analyze_market_sensitivity(base_params, solver_path, case_type, output_dir):
    # ... (此处省略，以保持简洁，使用您已有的函数)
    pass 

# --- 3. 新增的分析函数 ---
def analyze_production_sensitivity(base_params, solver_path, case_type, output_dir):
    """(2) 农业生产参数(亩产量)的敏感性分析"""
    print("\n" + "="*50)
    print("--- (2) 开始进行农业生产参数灵敏度分析 ---")
    
    # !!! 请选择一个您认为对气候敏感的作物 !!!
    SENSITIVE_CROP = '水稻'
    VARIATION_PERCENTAGES = np.arange(-0.20, 0.21, 0.05)
    yield_profits = []

    print(f"\n正在分析敏感作物 '{SENSITIVE_CROP}' 的亩产量影响...")
    original_yields = {key: val for key, val in base_params['P_yield'].items() if key[0] == SENSITIVE_CROP}
    if not original_yields:
        print(f"警告: 在数据中未找到作物 '{SENSITIVE_CROP}' 的产量信息，跳过此分析。")
        return

    for p in VARIATION_PERCENTAGES:
        temp_params = copy.deepcopy(base_params)
        for key in original_yields:
            if key in temp_params['P_yield']:
                temp_params['P_yield'][key] *= (1 + p)
        
        print(f"  -> 正在求解亩产量变动: {p:+.0%}")
        profit = solve_for_profit(temp_params, case_type, solver_path)
        yield_profits.append(profit)
    print(f" -> '{SENSITIVE_CROP}' 亩产量分析完成。")

    # 可视化
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(VARIATION_PERCENTAGES, yield_profits, marker='s', linestyle='-', color='#5A9BD4', label=f'{SENSITIVE_CROP}亩产量波动影响')
    ax.set_title('关键作物亩产量波动对经济效益的冲击评估', fontsize=16, pad=15)
    ax.set_xlabel('亩产量基准值变动百分比', fontsize=12)
    ax.set_ylabel('七年总利润 (元)', fontsize=12)
    ax.xaxis.set_major_formatter(mticker.PercentFormatter(xmax=1.0))
    ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0), useMathText=True)
    ax.grid(True, linestyle='--', linewidth=0.5)
    ax.legend(fontsize=10)
    fig.tight_layout()
    save_path = output_dir / 'figure_yield_sensitivity.png'
    plt.savefig(save_path, dpi=300)
    print(f"\n亩产量分析图表已成功保存至: {save_path}")
    plt.show()

def analyze_management_sensitivity(base_params, solver_path, case_type, output_dir):
    """(3) 管理决策超参数(最小种植面积)的敏感性分析"""
    print("\n" + "="*50)
    print("--- (3) 开始进行管理决策超参数灵敏度分析 ---")

    # 设置一系列要测试的 A_min 值 (单位: 亩)
    A_MIN_VALUES = [0.1, 0.2, 0.5, 0.8, 1.0, 1.2, 1.5]
    amin_profits = []
    
    print("\n正在分析最小种植面积 (Amin) 的影响...")
    for a_min in A_MIN_VALUES:
        # 注意：这里我们不需要复制params，因为改变的是模型结构参数
        print(f"  -> 正在求解 Amin = {a_min} 亩")
        # 直接将 a_min 传递给求解函数
        profit = solve_for_profit(base_params, case_type, solver_path, A_min=a_min)
        amin_profits.append(profit)
    print(" -> Amin 分析完成。")

    # 可视化
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(A_MIN_VALUES, amin_profits, marker='d', linestyle='--', color='#ED7D31', label='总利润随Amin变化')
    ax.set_title('最小种植面积(Amin)对总利润的影响', fontsize=16, pad=15)
    ax.set_xlabel('最小种植面积 Amin (亩)', fontsize=12)
    ax.set_ylabel('七年总利润 (元)', fontsize=12)
    ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0), useMathText=True)
    ax.grid(True, linestyle='--', linewidth=0.5)
    ax.legend(fontsize=10)
    fig.tight_layout()
    save_path = output_dir / 'figure_amin_sensitivity.png'
    plt.savefig(save_path, dpi=300)
    print(f"\nAmin 分析图表已成功保存至: {save_path}")
    plt.show()

# --- 主程序 ---
if __name__ == '__main__':
    try:
        # 配置Matplotlib
        plt.rcParams['font.sans-serif'] = ['SimHei']
        plt.rcParams['axes.unicode_minus'] = False

        # 路径设置
        current_dir = Path.cwd()
        project_root = current_dir.parent.parent
        path_f1 = project_root / 'Data' / '附件1.xlsx'
        path_f2 = project_root / 'Data' / '附件2.xlsx'
        output_dir = project_root / 'Code' / '1' / 'results'
        output_dir.mkdir(parents=True, exist_ok=True)
        solver_path = project_root / 'CBC' / 'bin' / 'cbc.exe'
        if not solver_path.exists():
            raise FileNotFoundError(f"错误：未找到求解器！请检查路径：{solver_path}")

        # 加载数据
        params = load_and_prepare_data(path_f1, path_f2)
        if params is None:
            raise RuntimeError("数据加载失败，无法继续。")
            
        # --- 基础问题求解 (可以注释掉以节省时间) ---
        print("\n--- 开始求解基础问题 ---")
        # build_and_solve_once(copy.deepcopy(params), 'waste', str(solver_path))
        # build_and_solve_once(copy.deepcopy(params), 'discount', str(solver_path))
        print("\n基础问题求解已跳过。") # 提示用户
        
        # --- 调用所有灵敏度分析 ---
        # 设定所有分析都基于 'discount' 情况
        analysis_case_type = 'discount'

        # (1) 市场经济参数分析 (您原来的函数，为保持一致性，建议也封装起来)
        # analyze_market_sensitivity(params, str(solver_path), analysis_case_type, output_dir)
        
        # (2) 农业生产参数分析
        analyze_production_sensitivity(params, str(solver_path), analysis_case_type, output_dir)

        # (3) 管理决策超参数分析
        analyze_management_sensitivity(params, str(solver_path), analysis_case_type, output_dir)

        print("\n所有灵敏度分析完成！")

    except Exception as e:
        print(f"\n程序运行出错: {str(e)}")
        import traceback
        traceback.print_exc()

 -> 数据参数准备完成。

--- 开始求解基础问题 ---

基础问题求解已跳过。

--- (2) 开始进行农业生产参数灵敏度分析 ---

正在分析敏感作物 '水稻' 的亩产量影响...

程序运行出错: name 'VARIAION_PERCENTAGES' is not defined


C:\Users\86198\AppData\Local\Temp\ipykernel_11276\577733929.py:74: UserWarning: 警告: 为163个作物-地块类型组合使用了默认值
  if missing_combinations > 0: warnings.warn(f"警告: 为{missing_combinations}个作物-地块类型组合使用了默认值")
Traceback (most recent call last):
  File "C:\Users\86198\AppData\Local\Temp\ipykernel_11276\577733929.py", line 298, in <module>
    analyze_production_sensitivity(params, str(solver_path), analysis_case_type, output_dir)
  File "C:\Users\86198\AppData\Local\Temp\ipykernel_11276\577733929.py", line 201, in analyze_production_sensitivity
    for p in VARIAION_PERCENTAGES:
NameError: name 'VARIAION_PERCENTAGES' is not defined
